# Kanana RAG Fine-tuning Notebook

This notebook fine-tunes the Kanana 8B instruct model on RAG tasks using the Jecheon tourism dataset.

**Model:** kakaocorp/kanana-1.5-8b-instruct-2505

**Training Data Format:**
```
[Instruction]
당신은 제천시 관광 안내 전문가입니다.
제공된 여러 문서 중에서 질문과 관련된 문서를 찾아, 그 문서의 내용을 바탕으로 정확하고 친절하게 답변해주세요.

Information:
{content1}
Information:
{content2}
Question: {question}
```

In [ ]:
# Install required packages
!pip install transformers peft datasets wandb bitsandbytes accelerate

In [ ]:
# Check CUDA availability
import os
import torch

os.environ["NVIDIA_VISIBLE_DEVICES"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"cuDNN version: {torch.backends.cudnn.version()}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Login to Weights & Biases
import wandb

# Login to wandb (you'll be prompted for your API key)
wandb.login()

# Initialize wandb project
wandb.init(
    project="kanana-rag-finetuning",
    name="kanana-1.5-8b-instruct-rag",
    config={
        "model": "kakaocorp/kanana-1.5-8b-instruct-2505",
        "task": "RAG fine-tuning",
        "dataset": "Jecheon Tourism"
    }
)

In [ ]:
# Load base model and tokenizer with QLoRA (4bit quantization)
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "kakaocorp/kanana-1.5-8b-instruct-2505"

# Configure 4bit quantization for QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Enable 4bit loading
    bnb_4bit_use_double_quant=True,         # Double quantization for better compression
    bnb_4bit_quant_type="nf4",              # NF4 quantization type (best for fine-tuning)
    bnb_4bit_compute_dtype=torch.bfloat16   # Compute in bfloat16 for stability
)

print(f"Loading model with QLoRA: {model_name}")
print(f"Quantization: 4bit NF4 with double quantization")

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Apply 4bit quantization
    trust_remote_code=True,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

print("Model loaded successfully with QLoRA!")
print(f"Model dtype: {base_model.dtype}")
print(f"Model device: {base_model.device}")

In [ ]:
# Load RAG training data
import json
from datasets import Dataset

data_path = "/home/user/goodganglabs/data/processed/training_data.jsonl"

# Load JSONL data
data_list = []
with open(data_path, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line))

print(f"Loaded {len(data_list)} training examples")
print("\nFirst example:")
print(json.dumps(data_list[0], ensure_ascii=False, indent=2)[:500])

In [ ]:
# Convert to requested RAG format with instruction
def format_rag_data(example):
    """Convert RAG data to the requested format with instruction:
    [Instruction]
    당신은 제천시 관광 안내 전문가입니다...
    
    Information:
    {content1}
    Information:
    {content2}
    Question: {question}
    """
    instruction = """당신은 제천시 관광 안내 전문가입니다.
제공된 여러 문서 중에서 질문과 관련된 문서를 찾아, 그 문서의 내용을 바탕으로 정확하고 친절하게 답변해주세요.

답변 시 주의사항:
1. 관련 문서의 내용만을 바탕으로 답변하세요
2. 문서에 정보가 없으면 "제공된 정보에는 해당 내용이 없습니다"라고 답변하세요
3. 추측하거나 문서 외부 지식을 사용하지 마세요
4. 간결하고 이해하기 쉽게 답변하세요"""
    
    documents = example['documents']
    question = example['question']
    answer = example['answer']
    
    # Build information sections
    info_sections = []
    for doc in documents:
        info_sections.append(f"Information:\n{doc['content']}")
    
    # Combine: instruction + information sections + question
    prompt = instruction + "\n\n"
    prompt += "\n\n".join(info_sections)
    prompt += f"\n\nQuestion: {question}"
    
    return {
        "prompt": prompt,
        "answer": answer
    }

# Apply formatting
formatted_data = []
for example in data_list:
    formatted_data.append(format_rag_data(example))

print(f"Formatted {len(formatted_data)} examples")
print("\nExample formatted prompt:")
print(formatted_data[0]['prompt'][:500])
print(f"\nAnswer: {formatted_data[0]['answer']}")

In [ ]:
# Create training dataset with proper format
def formatting_prompts_func(examples):
    """Format prompts for training"""
    prompts = examples["prompt"]
    answers = examples["answer"]
    
    EOS_TOKEN = tokenizer.eos_token
    
    texts = []
    for prompt, answer in zip(prompts, answers):
        # Combine prompt and answer with EOS token
        text = f"{prompt}\n\nAnswer: {answer}{EOS_TOKEN}"
        texts.append(text)
    
    return {"text": texts}

# Create dataset
dataset = Dataset.from_list(formatted_data)
dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"Dataset size: {len(dataset)}")
print(f"\nDataset features: {dataset.features}")
print(f"\nFirst training example:")
print(dataset[0]['text'][:500])

In [ ]:
# Tokenize dataset with reduced max_length for memory efficiency
def tokenize_function(examples):
    tokens = tokenizer(
        examples["text"], 
        padding="max_length",
        truncation=True,
        max_length=1024,  # Reduced from 2048 to save memory
        return_tensors="pt"
    )
    tokens["labels"] = tokens["input_ids"].clone()
    return tokens

tokenized_dataset = dataset.map(
    tokenize_function, 
    batched=True, 
    remove_columns=["text", "prompt", "answer"]
)

print(f"Tokenized dataset size: {len(tokenized_dataset)}")
print(f"Features: {tokenized_dataset.features}")
print(f"Max sequence length: 1024 tokens")

In [ ]:
# Configure LoRA for parameter-efficient fine-tuning with QLoRA optimizations
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for k-bit training (QLoRA requirement)
print("Preparing model for k-bit training...")
base_model = prepare_model_for_kbit_training(base_model)

# Enable gradient checkpointing to save memory
base_model.gradient_checkpointing_enable()
print("Gradient checkpointing enabled")

# Configure LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,                    # LoRA rank (increased from 8 for better performance)
    lora_alpha=32,          # LoRA alpha (scaling factor)
    lora_dropout=0.1,       # Dropout probability
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"           # Added o_proj for better coverage
    ],
    bias="none"
)

model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

# Log LoRA config to wandb
wandb.config.update({
    "quantization": "4bit NF4",
    "lora_r": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
    "lora_dropout": lora_config.lora_dropout,
    "target_modules": lora_config.target_modules,
    "gradient_checkpointing": True
})

In [ ]:
# Check GPU memory before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Log to wandb
wandb.config.update({
    "gpu_name": gpu_stats.name,
    "gpu_max_memory_gb": max_memory,
    "gpu_start_memory_gb": start_gpu_memory
})

In [ ]:
# Configure training arguments with memory-efficient settings
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    # Output settings
    output_dir="./outputs/kanana-rag",
    overwrite_output_dir=True,
    
    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=2,      # Can reduce to 1 if still OOM
    gradient_accumulation_steps=4,      # Effective batch size = 2*4 = 8
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=10,
    max_grad_norm=0.3,                  # Gradient clipping for stability
    
    # Memory optimization
    gradient_checkpointing=True,        # Critical for memory savings
    
    # Optimization
    bf16=True,
    optim="paged_adamw_8bit",          # Changed to 8bit optimizer for QLoRA
    lr_scheduler_type="linear",
    
    # Logging
    logging_steps=5,
    logging_dir="./logs",
    report_to="wandb",  # Enable wandb reporting
    
    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=3,
    
    # Evaluation
    evaluation_strategy="no",  # No validation set in this example
    
    # Other
    seed=1234,
    data_seed=1234,
    remove_unused_columns=True,
)

print("Training arguments configured:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size}")
print(f"  Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"  Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Optimizer: {training_args.optim}")
print(f"  Gradient checkpointing: {training_args.gradient_checkpointing}")
print(f"  Total steps: ~{len(tokenized_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

print("Trainer initialized successfully!")

In [ ]:
# Start training
print("Starting training...")
print("=" * 50)

trainer_stats = trainer.train()

print("\n" + "=" * 50)
print("Training completed!")
print(f"Training loss: {trainer_stats.training_loss:.4f}")
print(f"Training time: {trainer_stats.metrics['train_runtime']:.2f}s")

In [ ]:
# Check final GPU memory usage
final_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
print(f"\nFinal GPU memory: {final_gpu_memory} GB")
print(f"Peak memory used: {final_gpu_memory - start_gpu_memory} GB")

# Log final stats to wandb
wandb.log({
    "final_gpu_memory_gb": final_gpu_memory,
    "peak_memory_used_gb": final_gpu_memory - start_gpu_memory
})

In [ ]:
# Save the fine-tuned model
output_dir = "./outputs/kanana-rag-final"

print(f"Saving model to {output_dir}...")
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved successfully!")
print(f"\nModel location: {output_dir}")

In [ ]:
# Test inference with a sample
print("Testing inference...\n")

# Get a test example
test_prompt = formatted_data[0]['prompt']
expected_answer = formatted_data[0]['answer']

print("Test Prompt:")
print(test_prompt[:300])
print("\n...")

# Tokenize and generate
model.eval()
inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("\nGenerated Response:")
print(generated_text[len(test_prompt):])

print("\nExpected Answer:")
print(expected_answer)

In [ ]:
# Close wandb run
wandb.finish()
print("WandB run finished!")

## Summary

This notebook implements **QLoRA (Quantized LoRA)** fine-tuning for memory efficiency:

### Key Features:

1. ✅ **QLoRA (4bit quantization)**
   - NF4 quantization type
   - Double quantization enabled
   - Compute in bfloat16 for stability
   - **Memory savings: ~75% compared to full precision**

2. ✅ **Memory optimization techniques**
   - Gradient checkpointing enabled
   - Paged AdamW 8bit optimizer
   - Reduced max sequence length (1024 tokens)
   - Gradient clipping (max_grad_norm=0.3)

3. ✅ **LoRA configuration**
   - Rank: 16
   - Alpha: 32
   - Target modules: q_proj, k_proj, v_proj, o_proj
   - Dropout: 0.1

4. ✅ **Training format**
   ```
   [Instruction: 제천시 관광 안내 전문가 역할]
   
   Information:
   {content1}
   Information:
   {content2}
   Question: {question}
   
   Answer: {answer}
   ```

### Expected Memory Usage (A100 40GB):

- Model (4bit): ~2-3 GB
- Optimizer states: ~1-2 GB
- Gradients: ~1-2 GB
- Activations (with checkpointing): ~3-5 GB
- **Total: ~8-12 GB** (well within 40GB limit)

### Changes from Original:

| Setting | Before | After | Impact |
|---------|--------|-------|--------|
| Quantization | bfloat16 | 4bit NF4 | -75% memory |
| Max length | 2048 | 1024 | -50% activation memory |
| Optimizer | adamw_torch | paged_adamw_8bit | -50% optimizer memory |
| Gradient checkpoint | ❌ | ✅ | -40% activation memory |
| LoRA rank | 8 | 16 | +better performance |
| Target modules | 3 | 4 | +better coverage |

### Why This Should Work Now:

**Before (caused OOM):**
- 8B model × 2 bytes (bf16) = 16 GB base
- + optimizer states (32 GB)
- + gradients (16 GB)
- + activations (10-15 GB)
- **Total: 70-80 GB** ❌ Exceeds A100 capacity

**After (QLoRA):**
- 8B model × 0.5 bytes (4bit) = 4 GB base
- + optimizer states (2 GB, 8bit)
- + gradients (2 GB, LoRA only)
- + activations (4-6 GB, checkpointing)
- **Total: 12-14 GB** ✅ Fits comfortably

### Next Steps:
- Run training on A100
- Monitor GPU memory usage
- Evaluate fine-tuned model
- Compare baseline vs fine-tuned performance
- Upload model to Hugging Face Hub